<a href="https://colab.research.google.com/github/FarahnazHosseini/FarahnazHosseini/blob/main/GCNbyDGL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install dgl


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 35.2 MB/s eta 0:00:00


In [3]:
import os

os.environ["DGLBACKEND"] = "pytorch"
import dgl
import dgl.data
import torch
import torch.nn as nn
import torch.nn.functional as F

In [4]:
dataset = dgl.data.CoraGraphDataset()
print("Number of categories:", dataset.num_classes)

Extracting file to /root/.dgl/cora_v2
Finished data loading and preprocessing.
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.
Number of categories: 7


In [10]:
print(dataset)

Dataset("cora_v2", num_graphs=1, save_path=/root/.dgl/cora_v2)


In [5]:
g = dataset[0]

In [6]:
print("Node features")
print(g.ndata)
print("Edge features")
print(g.edata)

Node features
{'train_mask': tensor([ True,  True,  True,  ..., False, False, False]), 'val_mask': tensor([False, False, False,  ..., False, False, False]), 'test_mask': tensor([False, False, False,  ...,  True,  True,  True]), 'label': tensor([3, 4, 4,  ..., 3, 3, 3]), 'feat': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])}
Edge features
{}


In [7]:
#Defining a Graph Convolutional Network (GCN)
from dgl.nn import GraphConv


class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h


# Create the model with given dimensions
model = GCN(g.ndata["feat"].shape[1], 16, dataset.num_classes)

In [8]:
#Training the GCN
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata["feat"]
    labels = g.ndata["label"]
    train_mask = g.ndata["train_mask"]
    val_mask = g.ndata["val_mask"]
    test_mask = g.ndata["test_mask"]
    for e in range(100):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print(
                "In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})".format(
                    e, loss, val_acc, best_val_acc, test_acc, best_test_acc
                )
            )


model = GCN(g.ndata["feat"].shape[1], 16, dataset.num_classes)
train(g, model)

In epoch 0, loss: 1.946, val acc: 0.086 (best 0.086), test acc: 0.097 (best 0.097)
In epoch 5, loss: 1.893, val acc: 0.320 (best 0.344), test acc: 0.315 (best 0.371)
In epoch 10, loss: 1.816, val acc: 0.344 (best 0.344), test acc: 0.346 (best 0.371)
In epoch 15, loss: 1.717, val acc: 0.484 (best 0.484), test acc: 0.483 (best 0.483)
In epoch 20, loss: 1.595, val acc: 0.602 (best 0.602), test acc: 0.633 (best 0.633)
In epoch 25, loss: 1.452, val acc: 0.638 (best 0.638), test acc: 0.670 (best 0.670)
In epoch 30, loss: 1.292, val acc: 0.686 (best 0.686), test acc: 0.714 (best 0.714)
In epoch 35, loss: 1.121, val acc: 0.724 (best 0.724), test acc: 0.724 (best 0.724)
In epoch 40, loss: 0.951, val acc: 0.732 (best 0.732), test acc: 0.727 (best 0.723)
In epoch 45, loss: 0.791, val acc: 0.742 (best 0.742), test acc: 0.735 (best 0.735)
In epoch 50, loss: 0.648, val acc: 0.746 (best 0.746), test acc: 0.740 (best 0.741)
In epoch 55, loss: 0.525, val acc: 0.754 (best 0.754), test acc: 0.748 (best 0